In [21]:
# Newton's forward, backward and divided difference

import math
import numpy as np
import sympy as sp
from sympy import symbols, factorial, diff, lambdify, simplify

class Interpolation:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.forward_diff_row = []
        self.backward_diff_row = []
        self.divided_diff = []
        self.gauss_forward_diff = []
        self.gauss_backward_diff = []
        self.difference_table()
        self.u = symbols('u')   # for polynomial in u
        self.var = symbols('x') # real variable for polynomial

    def difference_table(self):
        curr_col = self.y.copy()
        curr_len = len(curr_col)
        
        while(curr_len > 1):
            for i in range (0, curr_len - 1):
                sub = curr_col[i + 1] - curr_col[i]
                if i == 0:
                    self.forward_diff_row.append(sub)
                if i == (curr_len - 2):
                    self.backward_diff_row.append(sub)
                curr_col.append(sub)
            curr_col = curr_col[curr_len:]
            curr_len = len(curr_col)  

    def calculating_u(self, u, n, is_forward):
        temp = 1
        if is_forward:
            for i in range(0, n):
                temp *= (u - i)
        else:
            for i in range(0, n):
                temp *= (u + i)
        return temp

    def newtons_forward_difference(self, x_point):
        h = self.x[1] - self.x[0]
        u = (x_point - self.x[0]) / h
        n = len(self.forward_diff_row)
        yx = self.y[0]
        for i in range(1, n + 1):  # +1 ensures the last difference (delta y0) is included
            yx = yx + (self.calculating_u(u, i, True) * self.forward_diff_row[i-1] / math.factorial(i))
        return yx

    def newtons_forward_difference_polynomial(self):
        h = self.x[1] - self.x[0]
        u = (self.var - self.x[0]) / h  # Takes into account 'x' as a variable symbol
        poly_expression = self.y[0]
        for i in range(1, len(self.forward_diff_row) + 1):
            term = 1
            for j in range(i):
                term *= (u - j)
            poly_expression += (term * self.forward_diff_row[i - 1]) / factorial(i)
        return simplify(poly_expr.expand())

    def newtons_backward_difference(self, x_point):
        h = self.x[1] - self.x[0]
        u = (x_point - self.x[len(self.x) - 1]) / h
        n = len(self.backward_diff_row)
        yx = self.y[len(self.y) - 1]
        for i in range(1, n + 1):  
            yx += (self.calculating_u(u, i, False) * self.backward_diff_row[i-1] / math.factorial(i))
        return yx

    def newtons_backward_difference_polynomial(self):
        h = self.x[1] - self.x[0]
        u = (self.var - self.x[-1]) / h
        poly_expression = self.y[-1]
        for i in range(1, len(self.backward_diff_row) + 1):
            term = 1
            for j in range(i):
                term *= (u + j)
            poly_expression += (term * self.backward_diff_row[i - 1]) / factorial(i)
        return simplify(poly_expression.expand())

    def calculating_divided_x(self, x_point, n):
        temp = 1.0
        for i in range(0, n):
            dif = x_point - self.x[i]
            if abs(dif) > 1e3:
                dif = np.sign(dif) * 1e3
            
            temp *= (dif)
            if abs(temp) > 1e308:
                temp = np.sign(temp) * 1e308
        return temp

    def newtons_divided_difference(self, x_point):
        n = len(self.y)

        # ------------------------

        if n > 10:
            diffs = np.abs(np.array(self.x) - x_point)
            idx = np.argsort(diffs)[:10]
            idx.sort()
            x_vals = [self.x[i] for i in idx]
            y_vals = [self.y[i] for i in idx]
        else:
            x_vals = self.x
            y_vals = self.y

        # ------------------------

        self.divided_diff = []
        curr_col = y_vals.copy()
        curr_x = x_vals.copy()
        curr_y = y_vals
        
        for i in range(1, len(curr_y)):
            for j in range(len(curr_y) - i):
                curr_col[j] = (curr_col[j + 1] - curr_col[j]) / (curr_x[i + j] - curr_x[j])
            self.divided_diff.append(curr_col[0])
        fx = y_vals[0]
        old_x = self.x
        self.x = x_vals 
        for i in range(len(self.divided_diff)):
            fx += self.calculating_divided_x(x_point, i + 1) * self.divided_diff[i]
        self.x = old_x
        return fx

    def divided_difference_table(self):
        n = len(self.y)
        self.divided_diff = []
        curr_col = self.y.copy()
        curr_x = self.x.copy()
        
        for i in range(1, n):
            for j in range(n - i):
                curr_col[j] = (curr_col[j + 1] - curr_col[j]) / (curr_x[i + j] - curr_x[j])
            self.divided_diff.append(curr_col[0])

    def newtons_divided_difference_polynomial(self):
        if not self.divided_diff:
            self.divided_difference_table()
        poly_expr = self.divided_diff[0]
        term = 1
        for i in range(1, len(self.x)):
            term *= (self.var - self.x[i - 1])
            poly_expr += self.divided_diff[i] * term
        return simplify(poly_expr.expand())

    def derivative_at_point(self, method="divided", x_val=0):
        if method == "forward":
            poly = self.newtons_forward_difference_polynomial()
        elif method == "backward":
            poly = self.newtons_backward_difference_polynomial()
        else:
            poly = self.newtons_divided_difference_polynomial()

        derivative = diff(poly, self.var)  # Third parameter is n, which gives order of derivativee
        f_prime_callable = lambdify(self.var, derivative, "numpy")  # Converts expressions given by sympy into callable functions using numpy
        return float(f_prime_callable(x_val)), poly, derivative

    def Lagrange_Interpolation(self, x_point):
        ratio = 1.0
        result = 0.0

        for i in range(len(self.x)):
            ratio = 1.0
            for j in range(len(self.x)):
                if i != j:
                    ratio *= (x_point - self.x[j]) / (self.x[i] - self.x[j])
            result += self.y[i] * ratio
        return result

    def stirling_coeff(self, p, i):
        coeff = 1.0
        if i%2 != 0:
            coeff *= p
            for k in range(1, (i + 1)//2):
                coeff *= ((p**2) - (k**2))
        else:
            coeff *= (p**2)
            for k in range(1, i//2):
                coeff *= ((p**2) - (k**2))
        return coeff / math.factorial(i)

    def Stirlings_central_difference(self, x_point):
        n = len(self.y)
        self.gauss_forward_diff = []
        self.gauss_backward_diff = []
        curr_col = self.y.copy()
        # curr_x = self.x.copy()
        midpoint_forward = midpoint_backward = n // 2
        
        for i in range(1, n):
            # Building the difference table column by column
            new_col = []
            for j in range(n - i):
                new_col.append(curr_col[j + 1] - curr_col[j])
            curr_col = new_col

            # Picking the correct central differences
            length = len(curr_col)
            f_index = b_index = 0
            if i % 2 != 0:  
                b_index = midpoint_backward - (i // 2) - 1  # Go upwards for odd order differences
            else:           
                b_index = midpoint_backward - (i // 2)  # Stay on the same index for even order differences

            # Backward and forward has common data at "even" intervals of delta_y
            f_index = midpoint_forward - (i // 2) # When index is odd, we stay on the same row, when even, move one step upwards
    
            # Checking whether indices are within bounds
            if 0 <= f_index < length:
                self.gauss_forward_diff.append(round(curr_col[f_index], 3))
            if 0 <= b_index < length:
                self.gauss_backward_diff.append(round(curr_col[b_index], 3))

        h = self.x[1] - self.x[0]
        p = (x_point - self.x[n // 2]) / h
        yx = self.y[n // 2]
        for i in range(1, len(self.gauss_forward_diff) + 1):
            # Averaging the odd terms
            diff = 0
            if i%2 != 0:
                diff = (self.gauss_forward_diff[i - 1] + self.gauss_backward_diff[i - 1]) / 2
            else:
                diff = self.gauss_forward_diff[i - 1]
                        
            coeff = self.stirling_coeff(p, i)
            yx += coeff * diff
        return yx

In [22]:
import numpy as np
import pandas as pd

# Newtons forward and backward interpolation is not suitable for large datasets, divided difference is suitable for such large datasets 

x = np.linspace(0, 100, 1000)  # Equally spaced 1000 points between 0 to 10
y = np.sin(x)

# with open("data.csv", "w") as data_file:
#     data_file.write("X, Y\n")
#     for i in range(len(x)):
#         data_file.write(f"{x[i]},{y[i]}\n")

# df = pd.read_csv("data.csv")

data = pd.DataFrame({"X" : x, "Y" : y})
data.to_csv("data.csv", index = False)

In [23]:
df = pd.read_csv("data.csv")
X = df["X"].to_list()
Y = df["Y"].to_list()

interpolationObj = Interpolation(X, Y)
nfd_result = interpolationObj.newtons_divided_difference(10)

nfd_result

-0.544021110889366

In [7]:
i1 = Interpolation([0.0, 0.1, 0.2, 0.3, 0.4], [1.0, 0.9975, 0.99, 0.9776, 0.8604])

# poly = i1.newtons_forward_difference_polynomial()
dy_dx, poly_expression, derivative_expression = i1.derivative_at_point(method = "backward", x_val = 0.4)
print(dy_dx)

-2.278999999999982


In [ ]:
# NumPy’s built-in linear/spline interpolators:
# from scipy.interpolate import interp1d

# f = interp1d(X, Y, kind="cubic")  # or "linear", "quadratic"
# print(f(3.5))

In [2]:
# # Newton's forward, backward and divided difference

# import math
# import numpy as np

# class Interpolation:
#     def __init__(self, x, y):
#         self.x = x
#         self.y = y
#         self.forward_diff_row = []
#         self.backward_diff_row = []
#         self.divided_diff = []
#         self.gauss_forward_diff = []
#         self.gauss_backward_diff = []
#         self.difference_table()
#         self.u = symbols('u')   # for polynomial in u
#         self.var = symbols('x') # real variable for polynomial

#     def difference_table(self):
#         # self.forward_diff_row = []
#         # self.backward_diff_row = []
#         curr_col = self.y.copy()
#         curr_len = len(curr_col)
        
#         while(curr_len > 1):
#             for i in range (0, curr_len - 1):
#                 sub = curr_col[i + 1] - curr_col[i]
#                 if i == 0:
#                     self.forward_diff_row.append(sub)
#                 if i == (curr_len - 2):
#                     self.backward_diff_row.append(sub)
#                 curr_col.append(sub)
#             curr_col = curr_col[curr_len:]
#             curr_len = len(curr_col)  

#     def calculating_u(self, u, n, is_forward):
#         temp = 1
#         if is_forward:
#             for i in range(0, n):
#                 temp *= (u - i)
#         else:
#             for i in range(0, n):
#                 temp *= (u + i)
#         return temp

#     def calculating_polynomial_u(self, n, is_forward):
#         temp = ""
#         if is_forward:
#             for i in range(0, n):
#                 if i == 0:
#                     temp = "u"
#                 else:
#                     temp += "(u - " + str(i) + ")"
#         else:
#             for i in range(0, n):
#                 temp += "(u + " + str(i) + ")"
#         return temp

#     def newtons_forward_difference(self, x_point):
#         h = self.x[1] - self.x[0]
#         u = (x_point - self.x[0]) / h
#         # self.difference_table()
#         n = len(self.forward_diff_row)
#         yx = self.y[0]
#         for i in range(1, n + 1):  # +1 ensures the last difference (delta y0) is included
#             yx = yx + (self.calculating_u(u, i, True) * self.forward_diff_row[i-1] / math.factorial(i))
#         return yx

#     def newtons_forward_difference_polynomial(self):
#         h = self.x[1] - self.x[0]
#         u = (self.var - self.x[0]) / h
#         poly_expr = self.y[0]
#         for i in range(1, len(self.forward_diff_row) + 1):
#             term = 1
#             for j in range(i):
#                 term *= (u - j)
#             poly_expr += (term * self.forward_diff_row[i - 1]) / factorial(i)
#         return simplify(poly_expr.expand())

#     def derivative_at_point(self, method="divided", x_val=0):
#         if method == "forward":
#             poly = self.newtons_forward_difference_polynomial()
#         elif method == "backward":
#             poly = self.newtons_backward_difference_polynomial()
#         else:
#             poly = self.newtons_divided_difference_polynomial()

#         derivative = diff(poly, self.var)
#         f_prime = lambdify(self.var, derivative, "numpy")
#         return float(f_prime(x_val)), poly, derivative

#     def newtons_backward_difference_polynomial(self):
#         h = self.x[1] - self.x[0]
#         u = (self.var - self.x[-1]) / h
#         poly_expr = self.y[-1]
#         for i in range(1, len(self.backward_diff_row) + 1):
#             term = 1
#             for j in range(i):
#                 term *= (u + j)
#             poly_expr += (term * self.backward_diff_row[i - 1]) / factorial(i)
#         return simplify(poly_expr.expand())

#     def newtons_backward_difference(self, x_point):
#         h = self.x[1] - self.x[0]
#         u = (x_point - self.x[len(self.x) - 1]) / h
#         # self.difference_table()
#         n = len(self.backward_diff_row)
#         yx = self.y[len(self.y) - 1]
#         for i in range(1, n + 1):  
#             yx += (self.calculating_u(u, i, False) * self.backward_diff_row[i-1] / math.factorial(i))
#         return yx

#     def divided_difference_table(self):
#         n = len(self.x)
#         table = [[0 for _ in range(n)] for __ in range(n)]
#         for i in range(n):
#             table[i][0] = self.y[i]
#         for j in range(1, n):
#             for i in range(n - j):
#                 table[i][j] = (table[i + 1][j - 1] - table[i][j - 1]) / (self.x[i + j] - self.x[i])
#         self.divided_diff = [table[0][j] for j in range(n)]
#         return table

#     def newtons_divided_difference_polynomial(self):
#         if not self.divided_diff:
#             self.divided_difference_table()
#         poly_expr = self.divided_diff[0]
#         term = 1
#         for i in range(1, len(self.x)):
#             term *= (self.var - self.x[i - 1])
#             poly_expr += self.divided_diff[i] * term
#         return simplify(poly_expr.expand())

In [1]:
# x = [0.0, 0.1, 0.2, 0.3, 0.4]
# y = [1.0, 0.9975, 0.99, 0.9776, 0.8604]
# # x = [2,4,9,10]
# # y = [4,56,711,980]

# interp = Interpolation(x, y)

# # Get polynomial (divided difference, works for unequal intervals too)
# poly = interp.newtons_divided_difference_polynomial()
# print("Polynomial:", poly)

# # Derivative and evaluation at x=2.5
# dy_val, poly_expr, derivative_expr = interp.derivative_at_point(method="divided", x_val=0)
# print("Polynomial expanded:", poly_expr)
# print("Derivative:", derivative_expr)
# print("dy/dx at x:", dy_val)